In [1]:
import sys
sys.path.insert(0, '../../../network')
import numpy as np
from sequences import GaussianSequence
from connectivity import cmatrix
from transfer_functions import ErrorFunction
from network import Population, RateNetwork

In [2]:
def zeros(size):
    return np.zeros(size, dtype=int)

def symmetric(size, val):
    M = np.zeros(size)
    for i in range(size[0]):
        for j in range(size[1]):
            if i == j:
                M[i][j] = val
    return M

def asymmetric(size, val):
    M = np.zeros(size)
    M[-1][0] = val
    for i in range(size[0]):
        for j in range(size[1]):
            if i+1 == j:
                M[i][j] = val
    return M

In [3]:
# Pop size
N_ctx, N_d1, N_d2 = 1500, 1500, 1500
N = [N_ctx, N_d1, N_d2]

# Gaussian patterns
S, P = 1, 4
sequences_ctx = [GaussianSequence(P,N_ctx, seed=1) for i in range(S)]
patterns_ctx = np.stack([s.inputs for s in sequences_ctx])
sequences_d1 = [GaussianSequence(P,N_d1, seed=127) for i in range(S)]
patterns_d1 = np.stack([s.inputs for s in sequences_d1])
sequences_d2 = [GaussianSequence(P,N_d2, seed=113) for i in range(S)]
patterns_d2 = np.stack([s.inputs for s in sequences_d2])

sequences = np.array([sequences_ctx, sequences_d1, sequences_d2], dtype=object)
patterns = np.array([patterns_ctx, patterns_d1, patterns_d2], dtype=object)

In [4]:
# Learning rule: 
# Type 0: zeros; 1: symmetric; 2: asymmetric
A00, A01, A02 = cmatrix((P,P), 5.1, type=1), cmatrix((P,P), 3, type=1), cmatrix((P,P), 0, type=0)
A00[0][0] = 5
A0 = np.array([A00,A01,A02])

A10, A11, A12 = cmatrix((P,P), .3, type=1), cmatrix((P,P), 5, type=1), cmatrix((P,P), 0, type=0) #.3
A1 = np.array([A10,A11,A12])

A20, A21, A22 = cmatrix((P,P), 0, type=0), cmatrix((P,P), 0, type=0), cmatrix((P,P), 0, type=0)
A2 = np.array([A20,A21,A22])

A = np.array([A0,A1,A2])

In [5]:
# Connection probabilities
cp = np.array([[0.2,0.05,0.05], 
               [0.05,0.1,0.2],
               [0.05,0.2,0.05]])
# Connection sign constraint
cw = np.array([[0,1,0],
               [0,-1,0],
               [0,0,-1]])

In [6]:
np.savez('./params.npz', 
         N=N, sequences=sequences, patterns=patterns,
         cp=cp, cw=cw, A=A)